In [11]:
from pathlib import Path

import pandas as pd
import numpy as np
import dtale
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

In [2]:
data_path = r"D:\AllenBrainObservatory\ephys_data"

In [3]:
def load_allen():
    data_directory = Path(data_path)
    manifest_path = data_directory / "manifest.json"
    cache = EcephysProjectCache.from_warehouse(
        manifest=manifest_path, fetch_tries=1, timeout=20000
    )
    return cache

In [4]:
cache = load_allen()

In [5]:
# Allen sdk help
help(cache)

Help on EcephysProjectCache in module allensdk.brain_observatory.ecephys.ecephys_project_cache object:

class EcephysProjectCache(allensdk.api.warehouse_cache.cache.Cache)
 |  EcephysProjectCache(fetch_api: Union[allensdk.brain_observatory.ecephys.ecephys_project_api.ecephys_project_api.EcephysProjectApi, NoneType] = None, fetch_tries: int = 2, stream_writer: Union[Callable, NoneType] = None, manifest: Union[str, pathlib.Path, NoneType] = None, version: Union[str, NoneType] = None, cache: bool = True)
 |  
 |  Method resolution order:
 |      EcephysProjectCache
 |      allensdk.api.warehouse_cache.cache.Cache
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, fetch_api: Union[allensdk.brain_observatory.ecephys.ecephys_project_api.ecephys_project_api.EcephysProjectApi, NoneType] = None, fetch_tries: int = 2, stream_writer: Union[Callable, NoneType] = None, manifest: Union[str, pathlib.Path, NoneType] = None, version: Union[str, NoneType] = None, cache: bool 

In [6]:
sessions = cache.get_session_table()

d = dtale.show(sessions)
# d.open_browser()
d

In [7]:
probes = cache.get_probes()

dtale.show(probes)

2022-04-07 15:22:16,415 - INFO     - Reading data from cache
2022-04-07 15:22:16,420 - INFO     - Reading data from cache
2022-04-07 15:22:16,427 - INFO     - Reading data from cache
2022-04-07 15:22:17,146 - INFO     - Reading data from cache
2022-04-07 15:22:17,245 - INFO     - Reading data from cache
2022-04-07 15:22:17,250 - INFO     - Reading data from cache
2022-04-07 15:22:17,401 - INFO     - Reading data from cache
2022-04-07 15:22:17,505 - INFO     - Reading data from cache
2022-04-07 15:22:17,509 - INFO     - Reading data from cache
2022-04-07 15:22:17,548 - INFO     - Reading data from cache
2022-04-07 15:22:17,648 - INFO     - Reading data from cache
2022-04-07 15:22:17,652 - INFO     - Reading data from cache


In [8]:
channels = cache.get_channels()
dtale.show(channels)

2022-04-07 15:22:46,442 - INFO     - Reading data from cache
2022-04-07 15:22:46,571 - INFO     - Reading data from cache
2022-04-07 15:22:46,575 - INFO     - Reading data from cache
2022-04-07 15:22:46,606 - INFO     - Reading data from cache
2022-04-07 15:22:47,446 - INFO     - Reading data from cache
2022-04-07 15:22:47,559 - INFO     - Reading data from cache
2022-04-07 15:22:47,563 - INFO     - Reading data from cache


In [9]:
units = cache.get_units()
dtale.show(units)

2022-04-07 15:23:18,866 - INFO     - Reading data from cache
2022-04-07 15:23:19,783 - INFO     - Reading data from cache
2022-04-07 15:23:19,916 - INFO     - Reading data from cache
2022-04-07 15:23:19,920 - INFO     - Reading data from cache


In [12]:
analysis_metrics1 = cache.get_unit_analysis_metrics_by_session_type(
    "brain_observatory_1.1",
    amplitude_cutoff_maximum=np.inf,
    presence_ratio_minimum=-np.inf,
    isi_violations_maximum=np.inf,
)

analysis_metrics2 = cache.get_unit_analysis_metrics_by_session_type(
    "functional_connectivity",
    amplitude_cutoff_maximum=np.inf,
    presence_ratio_minimum=-np.inf,
    isi_violations_maximum=np.inf,
)

all_metrics = pd.concat([analysis_metrics1, analysis_metrics2], sort=False)
dtale.show(all_metrics)

2022-04-07 15:25:58,236 - INFO     - Reading data from cache
2022-04-07 15:25:58,240 - INFO     - Reading data from cache
2022-04-07 15:25:59,086 - INFO     - Reading data from cache
2022-04-07 15:25:59,217 - INFO     - Reading data from cache
2022-04-07 15:25:59,223 - INFO     - Reading data from cache
2022-04-07 15:25:59,434 - INFO     - Reading data from cache
2022-04-07 15:25:59,549 - INFO     - Reading data from cache
2022-04-07 15:25:59,553 - INFO     - Reading data from cache
2022-04-07 15:25:59,600 - INFO     - Reading data from cache
2022-04-07 15:25:59,605 - INFO     - Reading data from cache
2022-04-07 15:25:59,617 - INFO     - Reading data from cache
2022-04-07 15:25:59,753 - INFO     - Reading data from cache
2022-04-07 15:25:59,758 - INFO     - Reading data from cache
2022-04-07 15:25:59,868 - INFO     - Reading data from cache
2022-04-07 15:26:01,147 - INFO     - Reading data from cache
2022-04-07 15:26:02,040 - INFO     - Reading data from cache
2022-04-07 15:26:02,171 

In [19]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

In [20]:
data_storage_directory = Path(r"D:\AllenBrainObservatory\ophys_data")
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

In [21]:
behavior_sessions = cache.get_behavior_session_table()
behavior_ophys_sessions = cache.get_ophys_session_table()
behavior_ophys_experiments = cache.get_ophys_experiment_table()

In [22]:
dtale.show(behavior_ophys_sessions)

In [23]:
dtale.show(behavior_ophys_experiments)

Executing shutdown due to inactivity...


2022-04-07 16:38:21,595 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-04-07 16:38:50,814 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "E:\virtualenvs\allensdk\lib\site-packages\dtale\app.py", line 410, in shutdown
    shutdown_server()
  File "E:\virtualenvs\allensdk\lib\site-packages\dtale\app.py", line 396, in shutdown_server
    raise RuntimeError("Not running with the Werkzeug Server")
RuntimeError: Not running with the Werkzeug Server


...

2022-04-07 18:52:22,982 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-04-07 18:52:51,704 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "E:\virtualenvs\allensdk\lib\site-packages\dtale\app.py", line 410, in shutdown
    shutdown_server()
  File "E:\virtualenvs\allensdk\lib\site-packages\dtale\app.py", line 396, in shutdown_server
    raise RuntimeError("Not running with the Werkzeug Server")
RuntimeError: Not running with the Werkzeug Server


2022-04-07 18:52:51,724 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "E:\virtualenvs\allensdk\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "E:\virtualenvs\allensdk\lib\site-packages\dtale\app.py", line 410, in shutdown
    shutdown_server()
  File "E:\virtualenvs\allensdk\lib\site-packages\dtale\app.py", line 396, in shutdown_server
    raise RuntimeError("Not running with the Werkzeug Server")
RuntimeError: Not running with the Werkzeug Server